In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

Couldn't import dot_parser, loading of dot files will not be possible.


In [2]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=False)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
def create_siamese_training_data(label):
    mnist_train = mnist.train.next_batch(10000)
    mnist_train_data = mnist_train[0]
    mnist_train_labels = mnist_train[1]
    #get all the mnist data with labels is 2
    mnist_two = mnist_train_data[mnist_train_labels==label]
    mnist_train_two = mnist_two[:-100]
    mnist_test_two = mnist_two[-100:]
    mnist_other_than_two = mnist_train_data[mnist_train_labels!=label]
    mnist_train_other_than_two = mnist_other_than_two[:-100]
    mnist_test_other_than_two = mnist_other_than_two[-100:]
    N_train = len(mnist_train_two)
    N_test = len(mnist_test_two)
    X_train_1 = np.concatenate((mnist_train_two,mnist_train_two),axis=0)
    Y_train_1 = np.ones(shape=(len(mnist_train_two)))
    X_train_2 = np.concatenate((mnist_train_two,mnist_train_other_than_two[0:N_train]),axis=0)
    Y_train_2 = np.zeros(shape=(len(mnist_train_other_than_two[0:N_train])))
    Y_train = np.concatenate((Y_train_1,Y_train_2))
    X_test_1 = np.concatenate((mnist_test_two,mnist_test_two),axis=0)
    Y_test_1 = np.ones(shape=(len(mnist_train_two)))
    X_test_2 = np.concatenate((mnist_test_two,mnist_test_other_than_two[0:N_test]),axis=0)
    Y_test_2 = np.zeros(shape=(len(mnist_test_other_than_two[0:N_test])))
    Y_test = np.concatenate((Y_test_1,Y_test_2))
    return X_train_1,X_train_2,Y_train,X_test_1,X_test_2,Y_test

In [4]:
def acc(label,predict):
    return np.mean(label==predict)

In [5]:
X_train_1,X_train_2,Y_train,X_test_1,X_test_2,Y_test = create_siamese_training_data(2)

In [6]:
tf.reset_default_graph()
tf_X1 = tf.placeholder(tf.float32, [None, 784])
tf_X2 = tf.placeholder(tf.float32, [None, 784])
tf_Y = tf.placeholder(tf.int32, [None])
W1_1 = tf.get_variable(name='W11',shape=(784,128),initializer=tf.random_normal_initializer())
b1_1 = tf.get_variable(name='b11',shape=(128),initializer=tf.zeros_initializer())
W1_2 = tf.get_variable(name='W12',shape=(784,128),initializer=tf.random_normal_initializer())
b1_2 = tf.get_variable(name='b12',shape=(128),initializer=tf.zeros_initializer())
W2 = tf.get_variable(name='W2',shape=(128,2),initializer=tf.random_normal_initializer())
b2 = tf.get_variable(name='b2',shape=(2),initializer=tf.zeros_initializer())

In [7]:
X_test_2.shape

(200, 784)

In [8]:
Z1_1 = tf.nn.relu(tf.add(tf.matmul(tf_X1,W1_1),b1_1))
Z1_2 = tf.nn.relu(tf.add(tf.matmul(tf_X2,W1_2),b1_2))
Z = tf.abs(tf.subtract(Z1_1,Z1_2))
logits = tf.add(tf.matmul(Z,W2),b2)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=tf.one_hot(tf_Y,2)))
train_op = tf.train.GradientDescentOptimizer(1e-4).minimize(loss)
predict_op = tf.argmax(tf.nn.softmax(logits),axis=1)
init_op = tf.global_variables_initializer()

In [9]:
with tf.Session() as sess:
    sess.run(init_op)
    for epoch in range(10000):
        _,l = sess.run([train_op,loss],feed_dict={
            tf_X1: X_train_1,
            tf_X2: X_train_2,
            tf_Y: Y_train
        })
        if epoch % 100 == 0:
            p_labels_train = sess.run(predict_op,feed_dict={
                tf_X1:X_train_1,
                tf_X2:X_train_2,
                tf_Y: Y_train
            })
            p_labels_test = sess.run(predict_op,feed_dict={
                tf_X1:X_test_1,
                tf_X2:X_test_2,
                tf_Y: Y_test
            })
            train_acc = acc(Y_train.reshape(-1,1),p_labels_train)
            test_acc = acc(Y_test.reshape(-1,1),p_labels_test)
            print("epoch: {} ----- loss: {} ------ train acc: {} ----- test acc: {}".format(epoch,l,train_acc,test_acc))
    

epoch: 0 ----- loss: 40.10014343261719 ------ train acc: 0.5 ----- test acc: 0.744
epoch: 100 ----- loss: 29.495107650756836 ------ train acc: 0.5 ----- test acc: 0.628
epoch: 200 ----- loss: 25.737777709960938 ------ train acc: 0.5 ----- test acc: 0.556
epoch: 300 ----- loss: 24.021730422973633 ------ train acc: 0.5 ----- test acc: 0.524
epoch: 400 ----- loss: 22.89626693725586 ------ train acc: 0.5 ----- test acc: 0.508
epoch: 500 ----- loss: 21.973880767822266 ------ train acc: 0.5 ----- test acc: 0.5
epoch: 600 ----- loss: 21.146835327148438 ------ train acc: 0.5 ----- test acc: 0.496
epoch: 700 ----- loss: 20.356935501098633 ------ train acc: 0.5 ----- test acc: 0.496
epoch: 800 ----- loss: 19.626808166503906 ------ train acc: 0.5 ----- test acc: 0.496
epoch: 900 ----- loss: 18.954050064086914 ------ train acc: 0.5 ----- test acc: 0.488
epoch: 1000 ----- loss: 18.337726593017578 ------ train acc: 0.5 ----- test acc: 0.484
epoch: 1100 ----- loss: 17.7769775390625 ------ train acc: 

epoch: 9600 ----- loss: 5.279184341430664 ------ train acc: 0.5 ----- test acc: 0.508
epoch: 9700 ----- loss: 5.235838890075684 ------ train acc: 0.5 ----- test acc: 0.508
epoch: 9800 ----- loss: 5.193149089813232 ------ train acc: 0.5 ----- test acc: 0.512
epoch: 9900 ----- loss: 5.151261329650879 ------ train acc: 0.5 ----- test acc: 0.512
